# Metadata

**Customer Domain:** - Entertainment Venues  
  
**Customer Subdomain:** - Movie Theater Management  
  
**Domain:** - Complex Instruction Following  
  
**Taxonomy Type:** - Tabular Data  
  
**L2 Taxonomy Type:** -  Data Transformation (Normalization/Standardization)  
  
**Conversation Length:** - 5 Turn Tasks  
  
**Scenario:** - [Movie Theater Chain] owns several cinemas across [State]. The management wants to compare the performance of different theaters to identify successful strategies and areas for improvement. However, the data from each theater is collected differently and stored in a non-uniform way. Each theater records data on ticket sales, concession sales, staffing levels, and movie screenings into CSV files. The format of the CSV files varies across locations – some use comma delimiters, others use semicolons, and the date formats are also inconsistent (e.g., "MM/DD/YYYY" or "YYYY-MM-DD").  
  
The [Regional Manager] needs a tool that can take this messy, string-encoded data and transform it into a standardized tabular format. This standardized table should include columns for [Theater ID], [Date], [Movie Title], [Number of Tickets Sold], [Concession Revenue], and [Staff On Duty]. The tool must handle different delimiters, inconsistent date formats, and string representations of numerical data. Furthermore, the user should be able to input the location and delimiter for each CSV file and specify the desired date format for the standardized output. The final transformed data will be used for comparative analysis and performance reporting using a [Business Intelligence software].  
  


**[system]**  
  
---  
  
You are an AI assistant designed to function as a data transformation tool for a Movie Theater Chain's Regional Manager. Your purpose is to parse, normalize, and standardize messy, string-encoded data representing sales and staffing information from different theater locations, which originally comes from CSV files with varying formats. The output should be a unified, clean tabular dataset suitable for performance analysis.  
  
Input Data Format and Validation:  
The user must first state the desired date format for the final output table (e.g., "Output date format should be YYYY-MM-DD"). Allowed output formats are "YYYY-MM-DD", "MM/DD/YYYY", "DD-Mon-YYYY".  
  
After specifying the output format, the user will provide the core input as a single string containing one or more pipe-separated (|) descriptions of the data sources. Each source description must contain exactly six comma-separated values in the following order:  
TheaterID_Str,FilePath_Str,Delimiter_Str,DateColumnName_Str,SourceDateFormat_Str,CSVContent_Str  
  
Where:  
  
TheaterID_Str is the identifier for the specific theater location (e.g., T101). (Data Type: String)  
  
FilePath_Str is the original file path for reference and must contain ".csv" (e.g., /data/theater1/sales_apr.csv). (Data Type: String)  
  
Delimiter_Str specifies the delimiter used within the CSVContent_Str. Must be one of: "comma", "semicolon". (Data Type: String)  
  
DateColumnName_Str is the exact header name of the date column within the CSVContent_Str. (Data Type: String)  
  
SourceDateFormat_Str specifies the format of dates within the date column of CSVContent_Str. Must be one of: "MM/DD/YYYY", "YYYY-MM-DD". (Data Type: String)  
  
CSVContent_Str is the raw CSV data provided as a multi-line string, including a header row. (Data Type: String)  
  
Input Validation:  
  
First, check if the user specified a valid target output date format ("YYYY-MM-DD", "MM/DD/YYYY", "DD-Mon-YYYY") before providing the data string. If not, request it.  
  
Next, validate the core input string. Check that it is non-empty and contains one or more pipe-separated segments. For each segment, validate that it contains exactly six comma-separated values. Validate the Delimiter_Str ("comma" or "semicolon"), SourceDateFormat_Str ("MM/DD/YYYY" or "YYYY-MM-DD"). Check that CSVContent_Str is present and contains at least a header row. Validate that FilePath_Str contains the substring ".csv".  
  
If any validation check fails, stop processing and return the following specific error message using Markdown formatting:  
  
```markdown  
**Error: Invalid Input Parameters**  
  
I could not process the request due to invalid input. Please ensure:  
1. You specified a valid target output date format ('YYYY-MM-DD', 'MM/DD/YYYY', or 'DD-Mon-YYYY') before the data string.  
2. The data string contains one or more pipe-separated (|) source descriptions.  
3. Each source description has exactly 6 comma-separated values: TheaterID, FilePath (containing '.csv'), Delimiter ('comma' or 'semicolon'), DateColumnName, SourceDateFormat ('MM/DD/YYYY' or 'YYYY-MM-DD'), CSVContent (including header).  
  
Example of correct format:  
User prompt: `Please process this data using YYYY-MM-DD output format.`  
Data string: `T101,/path/file1.csv,comma,SaleDate,MM/DD/YYYY,"Header1,Header2..."|T205,/path/file2.csv,semicolon,Date,YYYY-MM-DD,"HeaderA;HeaderB..."`  
  
Please correct the input and resubmit.  
```  
  
Data Transformation Logic:  
  
If the input is valid, perform the following data transformation steps:  
Initialize an empty list to hold standardized records and an empty list for potential data issue notes.  
  
For each pipe-separated FileDescription in the input string:  
Parse the TheaterID, FilePath, Delimiter, DateColumnName, SourceDateFormat, and CSVContent.  
  
Simulate reading the CSVContent string: Split into lines. Identify the header row (first line). Determine the column indices corresponding to the DateColumnName and the target output columns: Movie Title, Number of Tickets Sold, Concession Revenue, Staff On Duty. Assume standard header names like "Movie Title", "Movie", "Title", "Tickets Sold", "Tickets", "Concession Revenue", "Concessions", "Rev", "Staff On Duty", "Staff", "StaffCount". If essential columns are missing based on headers, skip this file description and add issue note: `[TheaterID] from [FilePath]: Skipped - Missing essential columns.`.  
  
Iterate through the data rows (lines after the header):  
Let CurrentRowNumber be the index+2.  
Parse the row using the specified Delimiter. Handle potential errors if a row has incorrect number of fields (add issue note: `[TheaterID] from [FilePath]: Row [CurrentRowNumber] skipped - incorrect field count.` and skip row).  
  
Extract the raw values for Date, Movie Title, Tickets Sold, Concession Revenue, and Staff On Duty based on their determined column indices.  
  
Normalize Date: Attempt to parse the date value using the specified SourceDateFormat. Convert the parsed date into the user-specified target output date format. Assume the year is the current year (2025) or the following year (2026) if the specified month has already passed in the current year (Current Date: April 21, 2025). If parsing fails, mark the date as 'Invalid Date'.  
Normalize Movie Title: Trim whitespace.  
Normalize Numerics: Attempt to convert 'Tickets Sold' and 'Staff On Duty' to integers. Attempt to convert 'Concession Revenue' to a floating-point number. If conversion fails, mark the value as 'ERROR'. Assume Concession Revenue is in USD. Check Tickets Sold: if successfully converted and value > 500, add issue note: `[TheaterID] from [FilePath]: Row [CurrentRowNumber] has high ticket count (>500) - potential data entry error.`  
  
Create a standardized record containing: TheaterID, FilePath, Normalized Date, Normalized Movie Title, Normalized Tickets Sold, Normalized Concession Revenue (USD), Normalized Staff On Duty.  
Add the standardized record to the results list.  
  
After processing all file descriptions, prepare the final output.  
  
Output Format:  
  
Present the transformed and standardized data as a Markdown table.  
Before the table, explicitly state the target date format used. Below the table, list any potential systemic data quality issues noted during processing.  
  
Example Output Structure:  
  
```markdown  
Target Date Format: [User-specified Target Date Format]  
  
| Theater ID | File Path Processed | Date       | Movie Title | Number of Tickets Sold | Concession Revenue (USD) | Staff On Duty |  
|------------|---------------------|------------|-------------|------------------------|--------------------------|---------------|  
| [TID]      | [FilePath]          | [Date]     | [Title]     | [Tickets]              | [Revenue]                | [Staff]       |  
| [TID]      | [FilePath]          | [Date]     | [Title]     | [Tickets]              | [Revenue]                | [Staff]       |  
| ...        | ...                 | ...        | ...         | ...                    | ...                      | ...           |  
  
Potential Data Issues Noted:  
* [Issue 1 Description, e.g., T101 from /path/file.csv: Row 5 has high ticket count (>500) - potential data entry error.]  
* [Issue 2 Description, e.g., T205 from /path/file2.csv: Skipped - Missing essential columns.]  
* (If no issues: None)  
```  
  
Constraints and Behavior:  
  
Process only the string data provided; do not assume access to actual files.  
Strictly follow the parsing, normalization, and standardization logic.  
Handle potential errors in CSV content or data types gracefully by marking data as 'ERROR' or noting skipped rows/files and specific potential issues below the table.  
Use "USD" suffix for the Concession Revenue column header.  
Conclude the interaction politely.  


**[user]**  
  
Process reports. Output format YYYY-MM-DD. `T777,/data/sales.log,comma,Day,YYYY-MM-DD,"Day,Title,Tix Sold,Rev,Staff\n2025-04-18,Blockbuster,250,800,7"`

**[assistant]**  
  
**Error: Invalid Input Parameters**  
  
I could not process the request due to invalid input. Please ensure:  
1. You specified a valid target output date format ('YYYY-MM-DD', 'MM/DD/YYYY', or 'DD-Mon-YYYY') before the data string.  
2. The data string contains one or more pipe-separated (|) source descriptions.  
3. Each source description has exactly 6 comma-separated values: TheaterID, FilePath (containing '.csv'), Delimiter ('comma' or 'semicolon'), DateColumnName, SourceDateFormat ('MM/DD/YYYY' or 'YYYY-MM-DD'), CSVContent (including header).  
  
The FilePath '/data/sales.log' for T777 does not contain '.csv'.  
  
Please correct the input and resubmit.

**[user]**  
  
Okay, fixed path. Output YYYY-MM-DD. `T777,/data/sales.csv,comma,Day,YYYY-MM-DD,"Day,Title,Tix Sold,Rev,Staff\n2025-04-18,Blockbuster,600,800,7\n2025-04-19,RomCom,abc,300,6"`

**[assistant]**  
  
Target Date Format: YYYY-MM-DD  
  
| Theater ID | File Path Processed | Date | Movie Title | Number of Tickets Sold | Concession Revenue (USD) | Staff On Duty |  
|------------|---------------------|------------|-------------|------------------------|--------------------------|---------------|  
| T777 | /data/sales.csv | 2025-04-18 | Blockbuster | 600 | 800.00 | 7 |  
| T777 | /data/sales.csv | 2025-04-19 | RomCom | ERROR | 300.00 | 6 |  
  
Potential Data Issues Noted:  
* T777 from /data/sales.csv: Row 2 has high ticket count (>500) - potential data entry error.  
* T777 from /data/sales.csv: Row 3 failed numeric conversion for Tix Sold.  
  
Data transformation complete. Let me know if you have more data to process.

**[user]**  
  
Yes, process T888 too. Keep YYYY-MM-DD. `T888,/data/daily.csv,semicolon,DateLogged,MM/DD/YYYY,"DateLogged;Movie Title;Tickets;Concession Revenue;Staff On Duty\n04/20/2025;Family Film;120;350.25;5"`

**[assistant]**  
  
Target Date Format: YYYY-MM-DD  
  
| Theater ID | File Path Processed | Date | Movie Title | Number of Tickets Sold | Concession Revenue (USD) | Staff On Duty |  
|------------|---------------------|------------|-------------|------------------------|--------------------------|---------------|  
| T888 | /data/daily.csv | 2025-04-20 | Family Film | 120 | 350.25 | 5 |  
  
Potential Data Issues Noted:  
* None  
  
Data transformation complete. Ready for the next one.

**[user]**  
  
Combine these two. Output MM/DD/YYYY. `T900,/path/fileA.csv,comma,DATE,YYYY-MM-DD,"DATE,TITLE,TICKETS,REVENUE,STAFF\n2025-04-21,Action Sequel,220,750,8"|T901,/path/fileB.csv,semicolon,Day Stamp,MM/DD/YYYY,"Day Stamp;Movie Name;Tickets Sold;;Staff\n04/21/2025;Indie Drama;75;;4"`

**[assistant]**  
  
Target Date Format: MM/DD/YYYY  
  
| Theater ID | File Path Processed | Date | Movie Title | Number of Tickets Sold | Concession Revenue (USD) | Staff On Duty |  
|------------|---------------------|------------|---------------|------------------------|--------------------------|---------------|  
| T900 | /path/fileA.csv | 04/21/2025 | Action Sequel | 220 | 750.00 | 8 |  
  
Potential Data Issues Noted:  
* T901 from /path/fileB.csv: Skipped - Missing essential columns (Concession Revenue based on header analysis).  
* T901 from /path/fileB.csv: Row 2 failed numeric conversion for Concession Revenue.  
  
Data transformation complete. Is there anything else?

**[user]**  
  
Last one. Output DD-Mon-YYYY. `T999,/data/final.csv,comma,Date,YYYY-MM-DD,"Date,Movie,Tickets Sold,Concession Revenue,Staff On Duty\n2025-04-18,Retro Flick,55,150.00,3\n2025-04-19,Retro Flick,65,175.50,3"`

**[assistant]**  
  
Target Date Format: DD-Mon-YYYY  
  
| Theater ID | File Path Processed | Date | Movie Title | Number of Tickets Sold | Concession Revenue (USD) | Staff On Duty |  
|------------|---------------------|-------------|-------------|------------------------|--------------------------|---------------|  
| T999 | /data/final.csv | 18-Apr-2025 | Retro Flick | 55 | 150.00 | 3 |  
| T999 | /data/final.csv | 19-Apr-2025 | Retro Flick | 65 | 175.50 | 3 |  
  
Potential Data Issues Noted:  
* None  
  
Data transformation complete. Feel free to provide more data when ready.